In [46]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from datetime import datetime
from collections import Counter

In [125]:
URL = "https://www.scoresway.com/en_GB/soccer/uefa-conference-league-2021-2022/kq8506uuyn3065k4fp8ni2vo/results"
FILE = r"D:\MetaData\UEFA\UEFA\UEFA_Conference_League_2021_2022_events.xlsx"

In [140]:
df = pd.read_excel(FILE)
df['event_url'] = df['event_url'].astype(str)
pattern = r'^(http|https|ftp)://[^\s]+$'

# Use df.query with @ to reference the pattern variable
df = df.query("event_url.str.match(@pattern)")
norm_teams = [ unidecode(x).lower() for x in set(df['home_team'].tolist()+df['away_team'].tolist())]
df['norm_home_team'] = [unidecode(x).lower() for x in df['home_team']]
df['norm_away_team'] = [unidecode(x).lower() for x in df['away_team']]
df['date'] = df['datetime'].dt.date
df['date'] = df['date'].astype(str)
df = df.sort_values('date',ascending=False)
df.head()

,home_team,away_team,home_score,away_score,datetime,event_url,norm_home_team,norm_away_team,date
0,AS Roma,Feyenoord Rotterdam,1,0,2022-05-26 00:30:00,https://www.scoresway.com/en_GB/soccer/uefa-co...,as roma,feyenoord rotterdam,2022-05-26
2,Olympique de Marseille,Feyenoord Rotterdam,0,0,2022-05-06 00:30:00,https://www.scoresway.com/en_GB/soccer/uefa-co...,olympique de marseille,feyenoord rotterdam,2022-05-06
1,AS Roma,Leicester City FC,1,0,2022-05-06 00:30:00,https://www.scoresway.com/en_GB/soccer/uefa-co...,as roma,leicester city fc,2022-05-06
3,Leicester City FC,AS Roma,1,1,2022-04-29 00:30:00,https://www.scoresway.com/en_GB/soccer/uefa-co...,leicester city fc,as roma,2022-04-29
4,Feyenoord Rotterdam,Olympique de Marseille,3,2,2022-04-29 00:30:00,https://www.scoresway.com/en_GB/soccer/uefa-co...,feyenoord rotterdam,olympique de marseille,2022-04-29


In [127]:
driver = webdriver.Chrome()

driver.get(URL)
fixture_list_locator = (By.CSS_SELECTOR, 'tbody.Opta-result.Opta-fixture')
all_locs = (By.CSS_SELECTOR, 'ul.Opta-TabbedContent')


In [133]:
from rapidfuzz import fuzz
from unidecode import unidecode
import re

def acronym(name: str) -> str:
    tokens = name.lower().split()
    return "".join(t[0] for t in tokens if t)

def normalize_team(name: str) -> str:
    name = unidecode(name).lower().strip()
    # Replace common non-alphanumeric separators with spaces
    name = re.sub(r'[^a-z0-9]+', ' ', name)
    return name

def best_fuzzy_match(team_name, choices, threshold_single=90, threshold_multi=75):
    q = normalize_team(team_name)
    q_tokens = q.split()
    single_word = len(q_tokens) == 1
    q_acr = acronym(q)
    results = {}

    for choice in choices:
        c = normalize_team(choice)
        c_acr = acronym(c)

        # Scores
        ts = fuzz.token_set_ratio(q, c)
        tsort = fuzz.token_sort_ratio(q, c)
        pr = fuzz.partial_ratio(q, c)
        score = max(ts, tsort, pr)

        # Acronym matching logic
        acr_score = 0
        if (len(q) <= 4 and q.isalpha()):  
            if q == c_acr:
                acr_score = 100
        else:
            if q_acr == c_acr and len(c_acr) <= 4:
                acr_score = 100

        score = max(score, acr_score)

        # Apply thresholds
        if single_word:
            if score >= threshold_single:
                results[choice] = score
        else:
            if score >= threshold_multi:
                results[choice] = score

    return dict(sorted(results.items(), key=lambda x: x[1], reverse=True))


In [146]:
# def best_fuzzy_match(team_name, choices, threshold=51):
#     # name,score = process.extractOne(team_name, choices, scorer=fuzz.token_sort_ratio)
#     response = process.extract(team_name, choices, scorer=fuzz.token_sort_ratio)
#     # print(name,score)
#     print(response)
#     # return None if score < threshold else name
fixtures = driver.find_elements(*fixture_list_locator)
max_matchs = len(fixtures)
miss = 1
print(f"{max_matchs} / {df.shape[0]}")
for i in range(max_matchs) :
    fixtures = driver.find_elements(*fixture_list_locator)
    fixture = fixtures[i]
    all_attributes = driver.execute_script(
        'var items = {}; for (index = 0; index < arguments[0].attributes.length; ++index) { items[arguments[0].attributes[index].name] = arguments[0].attributes[index].value }; return items;',
        fixture
    )
    w_home_team = fixture.find_element(By.CSS_SELECTOR,".Opta-Team.Opta-Home.Opta-TeamName").get_attribute("textContent")
    w_away_team = fixture.find_element(By.CSS_SELECTOR,".Opta-Team.Opta-Away.Opta-TeamName").get_attribute("textContent")
    ts = int(all_attributes.get('data-date',0))
    ts_sec = ts / 1000
    date_obj = datetime.fromtimestamp(ts_sec).strftime('%Y-%m-%d')
    
    f_home_team = best_fuzzy_match(w_home_team,norm_teams)
    f_away_team = best_fuzzy_match(w_away_team,norm_teams)
    
    filtered_df = df[
        df['norm_home_team'].str.lower().isin([t.lower() for t in f_home_team.keys()]) &
        df['norm_away_team'].str.lower().isin([t.lower() for t in f_away_team.keys()]) &
        df['date'].str.lower().isin([date_obj])
    ]
    if filtered_df.empty:
        print(f"Miss No {miss}\n"+
              f"HomeTeam = {w_home_team} | HomeVals = '{[t.lower() for t in f_home_team.keys()]}'\n"+
              f"AwayTeam = {w_away_team} | AwayVals = '{[t.lower() for t in f_away_team.keys()]}'\n"
              +
              f"Date = '{date_obj}'\n------------------")
        miss+=1
        # break
    else:
        print(filtered_df['date'].tolist()[0],date_obj,filtered_df['date'].tolist()[0] == date_obj)
        print(f"Worked For {w_home_team} v {w_away_team} @ {date_obj}\n")
    # all_teams
    # feyenoord rotterdam
    # feyenoord rotterdam

423 / 187
2022-05-26 2022-05-26 True
Worked For Roma v Feyenoord @ 2022-05-26

2022-05-06 2022-05-06 True
Worked For Roma v Leicester @ 2022-05-06

2022-05-06 2022-05-06 True
Worked For Marseille v Feyenoord @ 2022-05-06

2022-04-29 2022-04-29 True
Worked For Leicester v Roma @ 2022-04-29

2022-04-29 2022-04-29 True
Worked For Feyenoord v Marseille @ 2022-04-29

2022-04-15 2022-04-15 True
Worked For Slavia v Feyenoord @ 2022-04-15

2022-04-15 2022-04-15 True
Worked For Roma v Bodø/Glimt @ 2022-04-15

2022-04-15 2022-04-15 True
Worked For PAOK v Marseille @ 2022-04-15

2022-04-14 2022-04-14 True
Worked For PSV v Leicester @ 2022-04-14

2022-04-08 2022-04-08 True
Worked For Marseille v PAOK @ 2022-04-08

2022-04-08 2022-04-08 True
Worked For Leicester v PSV @ 2022-04-08

2022-04-08 2022-04-08 True
Worked For Bodø/Glimt v Roma @ 2022-04-08

2022-04-07 2022-04-07 True
Worked For Feyenoord v Slavia @ 2022-04-07

2022-03-18 2022-03-18 True
Worked For Roma v Vitesse @ 2022-03-18

2022-03-18 2

In [148]:
df[df['home_team'] == 'Rennes']

,home_team,away_team,home_score,away_score,datetime,event_url,norm_home_team,norm_away_team,date


In [132]:
from unidecode import unidecode
from rapidfuzz import fuzz

team_name = unidecode('Bodø/Glimt').lower()
choice = unidecode('FK Bodø/Glimt').lower()

ts = fuzz.token_set_ratio(team_name, choice)
tsort = fuzz.token_sort_ratio(team_name, choice)
pr = fuzz.partial_ratio(team_name, choice)

print(ts, tsort, pr)

100.0 86.95652173913044 100.0


In [118]:
miss

176

In [30]:
def get_all_match_info_at_once(driver, locator):
    try:
        # 1. Wait for the list of elements to be present
        elements = WebDriverWait(driver, 60).until(
            EC.presence_of_all_elements_located(locator)
        )
        # 2. Check if the index is valid
        if index >= len(elements):
            logger.info(f"Error: Index {index} is out of bounds for list of size {len(elements)}.")
            return False
                     
        item = elements[index]
        all_attributes = driver.execute_script(
        'var items = {}; for (index = 0; index < arguments[0].attributes.length; ++index) { items[arguments[0].attributes[index].name] = arguments[0].attributes[index].value }; return items;',
        item
        )
        # From Item, get home away team names - Done
        # make a utility function to give fuzzy results matching 60-70% like the name
        # Make all combination of the name
        # Check if anything like that is in DF. if yes skip
        # Else continue with execution
        clicker = item.find_element(By.CSS_SELECTOR, 'td.Opta-Divider.Opta-Dash[title="View match"]')
        driver.execute_script("arguments[0].click();", clicker)
        # element_to_click = elements[index]
        # element_to_click.click()
        
        logger.info(f"Successfully clicked element at index {index}.")
        return True # Exit function on success

    except WebDriverException as e:
        # 4. If a protocol error occurs, reload and prepare to retry
        if "ERR_HTTP2_PROTOCOL_ERROR" in str(e):
            logger.info(f"Attempt {attempt + 1}/{max_retries}: Protocol error detected. Reloading page...")
            driver.refresh()
        else:
            # For other errors, log it and fail immediately
            logger.info(f"An unhandled WebDriver error occurred: {e.__class__.__name__}")
            return False
            
    return False

{'class': 'Opta-fixture Opta-Match-1rncpj8trvlcnr74nvzstnehg Opta-result',
 'data-date': '1747854000000',
 'data-match': '1rncpj8trvlcnr74nvzstnehg',
 'data-match_winner_id': '22doj4sgsocqpxw45h607udje',
 'data-match_winner_side': 'home',
 'data-period': 'FullTime'}